# Week 01. 데이터 전처리 (Preprocessing)

| 항목 | 내용 |
|------|------|
| 과목 | 교육학 머신러닝 |
| 데이터 | Student Performance and Learning Behavior Dataset |
| 원본 출처 | [Zenodo (DOI: 10.5281/zenodo.16459132)](https://zenodo.org/records/16459132) |
| 라이선스 | CC BY 4.0 — 원저작자: Kamal Najem (Mohammed V University) |
| 변경사항 | 한글 변수명 변환, 일부 변수 삭제 및 재코딩 적용 |

## 0. 라이브러리 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

url = "https://raw.githubusercontent.com/SJ-EduLab/ML-for-Education/main/week01/student_learning_data_kr.csv"
df = pd.read_csv(url)

print(f"데이터 크기: {df.shape[0]}행 × {df.shape[1]}열")
df.head()

## 1. 데이터 탐색

전처리에 앞서 데이터의 구조와 특성을 파악합니다.

### 1-1. 변수 타입 및 결측치

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

### 1-2. 기술통계

In [ ]:
df.describe()

### 1-3. 범주형 변수 분포

In [ ]:
for col in df.select_dtypes(include='object').columns:
    print(f"\n[{col}]")
    print(df[col].value_counts())

## 2. 변수 유형 분류

각 변수의 특성에 따라 유형을 나눕니다. 이 분류가 이후 인코딩과 스케일링 방식을 결정합니다.

| 유형 | 변수 | 설명 |
|------|------|------|
| 연속형 | 주당학습시간, 출석률, 나이, 온라인강좌수, 과제완료율, 시험점수 | 숫자 자체에 크기·간격 의미가 있음 |
| 서열형 | 학습동기, 스트레스수준 | 순서에 의미가 있는 범주 (낮음 < 보통 < 높음) |
| 이진형 | 비교과활동, 인터넷접근, 성별, 토론참여 | 두 가지 값만 존재 |
| 명목형 | 선호학습유형 | 범주 간 순서 없음 (VARK 학습유형) |
| 타겟 | 최종성적 | 예측 대상 (A > B > C > D) |

In [ ]:
# 변수 유형 정의
continuous = ['주당학습시간', '출석률', '나이', '온라인강좌수', '과제완료율', '시험점수']
ordinal    = ['학습동기', '스트레스수준']
binary     = ['비교과활동', '인터넷접근', '성별', '토론참여']
nominal    = ['선호학습유형']
target     = '최종성적'

## 3. 서열형 변수 인코딩

'낮음', '보통', '높음'처럼 순서가 있는 범주는 그 순서를 반영하는 숫자로 변환합니다.

In [ ]:
ordinal_map = {'낮음': 0, '보통': 1, '높음': 2}

for col in ordinal:
    df[col] = df[col].map(ordinal_map)

df[ordinal].head()

## 4. 이진형 변수 인코딩

두 가지 값만 있는 변수는 0과 1로 변환합니다.

In [ ]:
binary_map = {
    '비교과활동': {'미참여': 0, '참여': 1},
    '인터넷접근': {'없음': 0, '있음': 1},
    '성별':       {'여': 0, '남': 1},
    '토론참여':   {'미참여': 0, '참여': 1},
}

for col, mapping in binary_map.items():
    df[col] = df[col].map(mapping)

df[binary].head()

## 5. 명목형 변수 인코딩 (원핫인코딩)

선호학습유형은 청각형·운동감각형·읽기쓰기형·시각형의 4개 범주입니다.

범주 간 순서가 없으므로, 각 범주를 독립 열로 분리합니다.

> **왜 원핫인코딩인가?**  
> 숫자(0,1,2,3)로 바꾸면 모델이 '시각형(3) > 청각형(0)'처럼  
> 존재하지 않는 크기 관계를 학습할 수 있기 때문입니다.

In [ ]:
df = pd.get_dummies(df, columns=nominal, prefix='학습유형')

style_cols = [c for c in df.columns if c.startswith('학습유형_')]
print(f"생성된 열: {style_cols}")
df[style_cols].head()

## 6. 타겟 변수 인코딩

최종성적(A/B/C/D)을 숫자로 변환합니다. 높은 숫자 = 좋은 성적이 되도록 코딩합니다.

In [ ]:
grade_map = {'D': 0, 'C': 1, 'B': 2, 'A': 3}
df[target] = df[target].map(grade_map)

df[target].value_counts().sort_index()

## 7. 연속형 변수 스케일링

연속형 변수들은 범위가 서로 다릅니다.
- 주당학습시간: 5 ~ 44
- 출석률: 60 ~ 100
- 나이: 18 ~ 29

StandardScaler를 적용하면 모든 변수가 **평균 0, 표준편차 1**로 통일되어,  
특정 변수가 모델에 과도한 영향을 미치는 것을 방지합니다.

In [ ]:
scaler = StandardScaler()
df[continuous] = scaler.fit_transform(df[continuous])

df[continuous].describe().round(2)

## 8. X, y 분리

전처리가 끝났으면 피처(X)와 타겟(y)을 분리합니다.

In [ ]:
y = df[target]
X = df.drop(columns=[target])

print(f"X: {X.shape}")
print(f"y: {y.shape}")
print(f"\ny 분포:")
print(y.value_counts().sort_index())

## 9. 전처리 결과 확인

In [ ]:
X.head()